In [28]:
import logging
%matplotlib inline
import os
import contextily as ctx
import partridge as ptg

from gtfs_router.raptor import raptor_assignment
from gtfs_router import log_stop_information
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug('Init Logger')

In [29]:
gtfs_path = os.path.join('data', 'sl')
#_date, service_ids = ptg.read_busiest_date(gtfs_path)
#view = {'routes.txt': {'service_id': service_ids}}
feed = ptg.load_geo_feed(gtfs_path)

#print('Date Selected: {}'.format(_date))

In [32]:
from gtfs_router.utils import generate_shape_dist_traveled

## Warnings are a result of route looping back on itself
new_stop_times = generate_shape_dist_traveled(feed, overwrite=True)

INFO:root:Processing Unique Trip Group 10 of 3215
INFO:root:Processing Unique Trip Group 20 of 3215
INFO:root:Processing Unique Trip Group 30 of 3215
INFO:root:Processing Unique Trip Group 40 of 3215
INFO:root:Processing Unique Trip Group 50 of 3215
INFO:root:Processing Unique Trip Group 60 of 3215
INFO:root:Processing Unique Trip Group 70 of 3215
INFO:root:Processing Unique Trip Group 80 of 3215
INFO:root:Processing Unique Trip Group 90 of 3215
/home/pablo/anaconda3/envs/geo_env/lib/python3.12/site-packages/shapely/linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
INFO:root:Processing Unique Trip Group 100 of 3215
INFO:root:Processing Unique Trip Group 110 of 3215
INFO:root:Processing Unique Trip Group 120 of 3215
INFO:root:Processing Unique Trip Group 130 of 3215
INFO:root:Processing Unique Trip Group 140 of 3215
INFO:root:Processing Unique Trip Group 150 of 3215
INFO:root:Processing Unique Trip Group 160 of 3215


In [33]:
## Additional field added 'shape_dist_traveled'
## Units are in meters based on default projection: Albers Equal Area Conical
new_stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled_x,timepoint,shape_dist_traveled_y
0,14010000331224227,23940.0,23940.0,9022001004201001,1,Nockeby,3,1,0.00,1,0.000000
1,14010000331224227,24000.0,24000.0,9022001004411001,2,Nockeby,3,3,864.43,1,861.931594
2,14010000331224227,24120.0,24120.0,9022001004421001,3,Nockeby,3,3,1329.66,1,1289.577973
3,14010000331224227,24240.0,24240.0,9022001004431001,4,Nockeby,3,3,2189.90,1,2169.148749
4,14010000331224227,24300.0,24300.0,9022001004441001,5,Nockeby,3,3,2741.63,1,2742.424959


In [26]:
# Depart at 8:30AM (in seconds)
departure_time = 8.5 * 60 * 60

# Up to two transfers are allowed
max_transfers = 3

# Stockholm West
from_stop_id = '9021001001281000' #Råcksta

#South of Stockholm
to_stop_id = '9021001000313000' #Henriksdals brygga

In [19]:
stop_state = raptor_assignment(feed, from_stop_id, to_stop_id, departure_time, feed.transfers, max_transfers)
print('Travel Time: {:.1f} minutes'.format(stop_state.get_stop(to_stop_id)['time_to_reach'] / 60))

INFO:root:No valid transfers found after iteration 0 for stop pair 9021001001281000->9021001000313000


KeyError: '9021001000313000'

In [17]:
paths = stop_state.describe_path(to_stop_id)

paths = paths.sort_index()

for idx, desc in zip(paths.index, paths['desc']):
    print('{}: {} \n'.format(idx + 1, desc))

ax = paths.to_crs(epsg=3857).plot(figsize=(10, 10), color='#FFFFFF', lw=8)
paths.to_crs(epsg=3857).plot(ax=ax, color=paths['color'], lw=4)

ctx.add_basemap(ax, zoom=14)

ax.set_axis_off()

KeyError: '9021001006061000'